In [1]:
import pyspark
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import *
from pyspark.sql.functions import lower, col
import re

In [2]:
sc = SparkContext()
sqlContext = SQLContext(sc)
spark = SparkSession \
        .builder \
        .appName("hw3") \
        .config("spark.some.config.option", "some-value") \
        .getOrCreate()


In [3]:
column_df = spark.read.load("/Users/tanyasah/Documents/BigData/PersonalProjects/github/personal/finalproject/projectdata/filtered/test2.tsv",
                     format="csv", delimiter="\t", inferSchema="true", header="true")                

In [4]:
us_cities_df = spark.read.load("/Users/tanyasah/Documents/BigData/PersonalProjects/github/personal/finalproject/projectdata/uscities/uscities.csv",
                     format="csv", delimiter=",", inferSchema="true", header="true")                

In [5]:
nyc_cities_df = us_cities_df.filter(us_cities_df.state_id=='NY').select('city')

In [6]:
us_street_df = spark.read.load("/Users/tanyasah/Documents/BigData/PersonalProjects/github/personal/finalproject/projectdata/usstreets/Street_Names.csv",
                     format="csv", delimiter=",", inferSchema="true", header="true") 

In [7]:
#loading the file for the street

In [8]:
street_column_df = spark.read.load("/Users/tanyasah/Documents/BigData/PersonalProjects/github/personal/finalproject/projectdata/filtered/bdjm-n7q4_street.tsv",
                     format="csv", delimiter="\t", inferSchema="true", header="true")       

In [9]:
#nyc_streets_df = us_streets_df.filter(us_cities_df.state_id=='NY').select('street')

### Phone number validation

In [10]:
phoneNumRegex = re.compile(r'(\d{3}[-\.\s]??\d{3}[-\.\s]??\d{4}|\(\d{3}\)\s*\d{3}[-\.\s]??\d{4}|\d{3}[-\.\s]??\d{4})')
def isPhoneNumber(text):
    if phoneNumRegex.match(text) != None:
        return True
    else: 
        return False

In [13]:
isPhoneNumber("(415) 555-4242")

True

In [14]:
#print('Phone number found: ' + str(a))
output_df = column_df.select("School Phone Number")
output_df.show(2)

+-------------------+
|School Phone Number|
+-------------------+
|        Unspecified|
|         7184301800|
+-------------------+
only showing top 2 rows



In [15]:
phone_udf = udf(isPhoneNumber, BooleanType())

In [16]:
output_df = output_df.withColumn('isPhoneNumber', phone_udf('School Phone Number'))

In [17]:
output_df.show()

+-------------------+-------------+
|School Phone Number|isPhoneNumber|
+-------------------+-------------+
|        Unspecified|        false|
|         7184301800|         true|
|        Unspecified|        false|
|        Unspecified|        false|
|        Unspecified|        false|
|        Unspecified|        false|
|        Unspecified|        false|
|        Unspecified|        false|
|        Unspecified|        false|
|         7187201010|         true|
|        Unspecified|        false|
|        Unspecified|        false|
|        Unspecified|        false|
|        Unspecified|        false|
|        Unspecified|        false|
|        Unspecified|        false|
|        Unspecified|        false|
|        Unspecified|        false|
|        Unspecified|        false|
|        Unspecified|        false|
+-------------------+-------------+
only showing top 20 rows



### City validation

In [18]:
nyc_cities_df.show(5)

+----------------+
|            city|
+----------------+
|     Port Jervis|
|   Brewster Hill|
|           Sodus|
|          Roslyn|
|Washington Mills|
+----------------+
only showing top 5 rows



In [16]:
lower_udf = udf(lambda x: x.lower(), StringType())

In [17]:
udf_1 = udf(lambda x: 1, IntegerType())

In [21]:
nyc_cities_df = nyc_cities_df.withColumn('city', lower_udf('city'))

nyc_cities_df = nyc_cities_df.withColumn('city_bool', udf_1('city'))

nyc_cities_df.show(5)

+----------------+---------+
|            city|city_bool|
+----------------+---------+
|     port jervis|        1|
|   brewster hill|        1|
|           sodus|        1|
|          roslyn|        1|
|washington mills|        1|
+----------------+---------+
only showing top 5 rows



In [22]:
output_df = column_df.select('School City')

output_df = output_df.withColumn('School City', lower_udf('School City'))

In [23]:
output_df.show(5)

+-----------+
|School City|
+-----------+
|unspecified|
|      bronx|
|unspecified|
|unspecified|
|unspecified|
+-----------+
only showing top 5 rows



In [24]:
test = output_df.join(nyc_cities_df, output_df['School City'] == nyc_cities_df['city'], how='left')

test =test.drop(test.city)

In [25]:
test.show(5)

+-----------+---------+
|School City|city_bool|
+-----------+---------+
|unspecified|     null|
|      bronx|        1|
|unspecified|     null|
|unspecified|     null|
|unspecified|     null|
+-----------+---------+
only showing top 5 rows



In [26]:
udf_city_true = udf(lambda x : True if x == 1 else False)

In [27]:
test =test.withColumn("city_present", udf_city_true('city_bool'))

In [28]:
test.show(5)

+-----------+---------+------------+
|School City|city_bool|city_present|
+-----------+---------+------------+
|unspecified|     null|       false|
|      bronx|        1|        true|
|unspecified|     null|       false|
|unspecified|     null|       false|
|unspecified|     null|       false|
+-----------+---------+------------+
only showing top 5 rows



### Street validation

In [11]:
us_street_df.show(5)

+--------------+-----------------------+------------------+-------------+------------------+-----------------------+----------------+----------------+-------------------+--------+--------------+-------------+
|STREET NAME ID|STREET PREFIX DIRECTION|STREET PREFIX TYPE|  STREET NAME|STREET SUFFIX TYPE|STREET SUFFIX DIRECTION|STREET EXTENSION|POSTAL COMMUNITY|MUNICIPAL COMMUNITY|NOTATION|PRIVATE STREET|FRONTAGE ROAD|
+--------------+-----------------------+------------------+-------------+------------------+-----------------------+----------------+----------------+-------------------+--------+--------------+-------------+
|             1|                      N|              null|          4TH|                ST|                   null|            null|     BATON ROUGE|        BATON ROUGE|    null|            NO|           NO|
|            10|                      S|              null|         11TH|                ST|                   null|            null|     BATON ROUGE|        BATON 

In [12]:
output_street= street_column_df.select("StreetName")

In [25]:
street_text_df = sc.textFile("/Users/tanyasah/Documents/BigData/PersonalProjects/github/personal/finalproject/projectdata/filtered/4twk-9yq2.CrossStreet2.txt")
  

In [18]:
us_street_df = us_street_df.withColumn('STREET NAME', lower_udf('STREET NAME'))

output_street = output_street.withColumn('StreetName', lower_udf('StreetName'))


### LATITUDE-LONGITUDE



In [29]:
lines =sc.textFile("/Users/tanyasah/Documents/BigData/PersonalProjects/github/personal/finalproject/projectdata/filtered/qgea-i56i.Lat_Lon.txt")
  
new =lines.map(lambda s: s.split("\t"))
new =new.map(lambda f: (f[0].replace("(","").replace(")",""), int(f[1])))
new.take(5)
dd = spark.createDataFrame(new, ['lat_long', 'count'])
dd.show(5)

#column_df = spark.read.load("/Users/tanyasah/Documents/BigData/PersonalProjects/github/personal/finalproject/projectdata/filtered/qgea-i56i.Lat_Lon.txt",
                    # format="csv", inferSchema="true", header="true")                

+--------------------+-----+
|            lat_long|count|
+--------------------+-----+
|40.112709974, -77...|    1|
|40.494426993, -77...|    1|
|40.498905363, -74...|   24|
|40.498957012, -74...|    9|
|40.499025362, -74...|   26|
+--------------------+-----+
only showing top 5 rows



In [30]:
latLongRegex = re.compile(r'(^[-+]?([1-8]?\d(\.\d+)?|90(\.0+)?),\s*[-+]?(180(\.0+)?|((1[0-7]\d)|([1-9]?\d))(\.\d+)?)$)')
def isLatLongitude(text):
    if latLongRegex.match(text) != None:
        return True
    else: 
        return False

In [31]:
isLatLongitude("+90.0, -127.554334")

True

In [42]:

lat_udf = udf(isLatLongitude, BooleanType())

output_df = dd.select("lat_long")

output_df = output_df.withColumn('isLatLongitude', lat_udf('lat_long'))

#output_df.show(5)

output_df.rdd.map( lambda x: x[1]== 'false').collect()

[False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,


### Sherlock POC